# Find the comparables: fixtures.txt

The file `fixtures.txt` contains important property features like number of bedrooms, full baths, half baths, and more. It comes as a melted table, so we need to use the pivot_table method on the dataframe instance to shape it to a table with one observation per row (account number).

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import pickle

import pandas as pd

from src.definitions import ROOT_DIR
from src.data.utils import Table, save_pickle

In [ ]:
fixtures_fn = ROOT_DIR / 'data/external/2016/Real_building_land/fixtures.txt'
assert fixtures_fn.exists()

In [ ]:
fixtures = Table(fixtures_fn, '2016')

In [ ]:
fixtures_df = fixtures.get_df()

In [ ]:
fixtures_df.head()

# Load accounts of interest
Let's remove the account numbers that don't meet free-standing single-family home criteria that we found while processing the `building_res.txt` file.

In [ ]:
one_bld_in_acct_fn = ROOT_DIR / 'data/raw/2016/one_bld_in_acct.pickle'

In [ ]:
with open(one_bld_in_acct_fn, 'rb') as f:
    one_bld_in_acct = pickle.load(f)

In [ ]:
cond0 = fixtures_df['acct'].isin(one_bld_in_acct)
fixtures_df = fixtures_df.loc[cond0, :]

In [ ]:
fixtures_df.head()

In [ ]:
fixtures_df['type_dscr'].value_counts()

# Select columns and build pivot table
From the value count on the fixtures type description above we can tell that the first 10 types are prevalent in the data. Let's focus on these 10 in our evaluation.

In [ ]:
cols = fixtures_df['type_dscr'].value_counts().head(10).index

In [ ]:
cond0 = fixtures_df['type_dscr'].isin(cols)
fixtures_df = fixtures_df.loc[cond0, :]

In [ ]:
fixtures_pivot = fixtures_df.pivot_table(index='acct', columns='type_dscr', values='units', fill_value=0)

In [ ]:
fixtures_pivot.head()

add `acct` column to make easier the merging process ahead

In [ ]:
fixtures_pivot.reset_index(inplace=True)

# Export fixtures_pivot

In [ ]:
save_fn = ROOT_DIR / 'data/raw/2016/fixtures_comps.pickle'
save_pickle(fixtures_pivot, save_fn)